In [27]:
import ROOT as r
import pandas as pd
import matplotlib.pyplot as plt
import mplhep as hep
import math
import numpy as np
#import colorsys
#from scipy import optimize as sciopt
#import os
import lmfit
import dataset as ds
import datasetXtals as dsX
import fit_functions as funcs
import importlib

In [28]:
plt.style.use(hep.style.ATLAS)
plt.style.use({"axes.labelsize":'36', 'lines.linewidth':'2', 'lines.markersize':'2', 'xtick.labelsize':'16'})

### Set up the DataSet objects

In [10]:
# Reload stuff in case you make changes!

importlib.reload(ds)
importlib.reload(dsX)
importlib.reload(funcs)

<module 'fit_functions' from '/home/lars/g-2/earlyPosition/fit_functions.py'>

In [48]:
version = "3"

for eLow in range(0, 100, 100):
    energy_lim = [1000, 3200]

    low_E  = str(energy_lim[0] / 1000)
    high_E = str(energy_lim[1] / 1000)
    
    #fit_stuff(setup_stuff())
    plot_stuff()

#### Clusters

#### Xtals

In [29]:
def setup_stuff():

    sixtyhourX = dsX.DataSetX("1A", "60h", r.TFile("./data/60hXtalsPosition_v"+version+".root"), energy_lim)
    highkickX  = dsX.DataSetX("1B", "HighKick", r.TFile("./data/highKickXtalsPosition_v"+version+".root"), energy_lim)
    ninedayX   = dsX.DataSetX("1C", "9day", r.TFile("./data/9dayXtalsPosition_v"+version+".root"), energy_lim)
    endgameX   = dsX.DataSetX("1D", "EndGame", r.TFile("./data/endGameXtalsPosition_v"+version+".root"), energy_lim)
    run2CX     = dsX.DataSetX("2C", "run2C", r.TFile("./data/run2CXtalsPosition_v"+version+".root"), energy_lim)

    datasetsX = {"1A": sixtyhourX, "1B":highkickX, "1C":ninedayX, "1D":endgameX, "2C": run2CX}

    for name, datasetX in datasetsX.items():
        datasetX.set_up_calos(step_length=10, verbose=False)
    return datasetsX

# Try Fitting with just a linear function

#### Play with a single fit

#### Fit the Stuff

In [30]:
def fit_stuff(datasetsX):

    hints = {
        'c':    [0, 50],
        'm':    [-1, 1]
    }

    firstest_guess = {
        'c':    35,
        'm':    0
    }

    names = ['1A', '1B', '1C', '1D', '2C']

    for name in names:

        this_data = pd.DataFrame(columns = ['c', 'm', 'c_err', 'm_err', 'redchi', 'caloNum'])

        for num, calo in datasetsX[name].calos.items():
            first_guess = firstest_guess
            for attempt in range(1, 11):
                try:
                    if(name == '1D'):
                        calo.build_params_0([50, 300], first_guess, hints)
                    else:
                        calo.build_params_0([30, 300], first_guess, hints)
                    if(calo.func0_params['randomized'] == {}):
                        first_guess = {'c':    33, 'm':    0}
                    else:
                        first_guess = calo.func0_params['randomized']

                except:
                    print("The fit failed for {0} calo {1} on attempt {2}".format(name, num, attempt))
                    first_guess = firstest_guess


            try:
                #print(p)
                p = calo.func0_params['randomized']
                p['caloNum'] = num
                this_data.loc[num] = p
            except:
                continue

            #print(this_data)
            this_data.to_csv("./csvs_of_fits/" + name + "_"+ low_E + "-" + high_E + "_GeV_v" + version + ".csv")

            #rand_data[name] = this_data

#### Plot the Fits

In [47]:
def plot_stuff():

    rand_data = {}
    names = ['1A', '1B', '1C', '1D', '2C']

    for name in names:
        rand_data[name] = pd.read_csv("./csvs_of_fits/" + name + "_"+ low_E + "-" + high_E + "_GeV_v" + version + ".csv")



    fig, ax = plt.subplots(1,3)
    fig.set_size_inches(30, 10)


    rand_data_2 = {}
    rand_data_2['1A'] = rand_data['1A']
    rand_data_2['1B'] = rand_data['1B']
    rand_data_2['1C'] = rand_data['1C']
    rand_data_2['1D'] = rand_data['1D']
    rand_data_2['2C'] = rand_data['2C']

    for name, df in rand_data_2.items():
        ax[0].errorbar(df['caloNum'], df['c'], yerr=df['c_err'], label=name)
        #ax[1].errorbar(df['caloNum'], df['m'], yerr=df['m_err'], label=name)
        ax[1].plot(df['caloNum'], df['m'], label=name)
        ax[2].plot(df['caloNum'], df['redchi'], label=name)
        
    ax[0].grid()
    ax[1].grid()
    ax[2].grid()

    ax[0].set_xlabel('calo number')    
    ax[1].set_xlabel('calo number')
    ax[2].set_xlabel('calo number')

    ax[0].set_ylabel('c    [mm]')
    ax[2].set_ylabel(r'$chi^{2}$', fontsize=40)
    ax[1].set_ylabel('m    [mm/μs]')
    
    fig.tight_layout()
    fig.suptitle("Crystals y-RMS Fits" + "        "+ low_E + "-" + high_E + " GeV from Cluster", fontsize=40, y=1.05)
    ax[2].legend(loc=1, labelspacing=1.5, framealpha=1);
    
    ax[2].set_ylim(0, 1.5)

    plt.savefig("./images/final_presentation/xtal_params" + "_" + low_E + "-" + high_E + "GeV_v" + version + "_uncut_nobars.png", bbox_inches='tight')

    ax[0].set_ylim(30, 36)
    ax[1].set_ylim(-0.0003, 0.0001)
    ax[2].set_ylim(0, 1.5)

    plt.savefig("./images/final_presentation/xtal_params" + "_" + low_E + "-" + high_E + "GeV_v" + version + "_cut_nobars.png", bbox_inches='tight')
    plt.close()

In [92]:
def plot_histogram():
    rand_data = {}
    names = ['1A', '1B', '1C', '1D', '2C']

    for name in names:
        rand_data[name] = pd.read_csv("./csvs_of_fits/" + name + "_"+ low_E + "-" + high_E + "_GeV_v" + version + ".csv")

    fig, ax = plt.subplots(1, 5)
    fig.set_size_inches(50, 10)
    
    for i, dname in enumerate(rand_data):
        ax[i].hist(rand_data[dname]['m'], bins=15, range=(-0.0003, 0.0001), color='C0');
        ax[i].set_title(dname, fontsize=40);
        ax[i].set_xlabel('m   [mm/μs]');
        ax[i].ticklabel_format(axis='x', style='sci', scilimits=(1, -5));
        ax[i].vlines(0, 0, 24, color='C3', linestyles='dashed', linewidth=2, label='m=0 reference line');
        ax[i].vlines(rand_data[dname]['m'].mean(), 0, 24, color='C2', linestyles='dashed', linewidth=2, label='mean value of m');
        ax[i].set_xlim(-0.0003, 0.0001);
        ax[i].set_ylim(0, 12);
        ax[i].legend(loc=(0, 1.5), labelspacing=1.5, framealpha=1);
    
    
    plt.savefig("./images/histogram/hist" + "_" + low_E + "-" + high_E + "GeV_v" + version + ".png", bbox_inches='tight');
    plt.close();

In [93]:
plot_histogram()

## Func 1